#### First import OS package in order to use enviroment variables

In [1]:
import os

#### Next, get the OpenAI package and set the Pandas package as pd

In [6]:
from openai import OpenAI

In [7]:
import pandas as pd

#### Get the config file with the API token and setup the client

In [10]:
import config
config.apiToken()

In [11]:
client = OpenAI()

In [13]:
papers = pd.DataFrame({
    "filename": [
        "2405.10313v1.pdf",
        "2401.03428v1.pdf",
        "2401.09395v2.pdf",
        "2401.13142v3.pdf",
        "2403.02164v2.pdf",
        "2403.12107v1.pdf",
        "2404.10731v1.pdf",
        "2312.11562v5.pdf",
        "2311.02462v2.pdf",
        "2310.15274v1.pdf"
    ],
    "title": [
        "How Far Are We From AGI?",
        "EXPLORING LARGE LANGUAGE MODEL BASED INTELLIGENT AGENTS: DEFINITIONS, METHODS, AND PROSPECTS",
        "CAUGHT IN THE QUICKSAND OF REASONING, FAR FROM AGI SUMMIT: Evaluating LLMs’ Mathematical and Coding Competency through Ontology-guided Interventions",
        "Unsocial Intelligence: an Investigation of the Assumptions of AGI Discourse",
        "Cognition is All You Need The Next Layer of AI Above Large Language Models",
        "Scenarios for the Transition to AGI",
        "What is Meant by AGI? On the Definition of Artificial General Intelligence",
        "A Survey of Reasoning with Foundation Models",
        "Levels of AGI: Operationalizing Progress on the Path to AGI",
        "Systematic AI Approach for AGI: Addressing Alignment, Energy, and AGI Grand Challenges"
    ]
})
papers["filename"] = "papers/" + papers["filename"]
papers

,filename,title
0,papers/2405.10313v1.pdf,How Far Are We From AGI?
1,papers/2401.03428v1.pdf,EXPLORING LARGE LANGUAGE MODEL BASED INTELLIGE...
2,papers/2401.09395v2.pdf,"CAUGHT IN THE QUICKSAND OF REASONING, FAR FROM..."
3,papers/2401.13142v3.pdf,Unsocial Intelligence: an Investigation of the...
4,papers/2403.02164v2.pdf,Cognition is All You Need The Next Layer of AI...
5,papers/2403.12107v1.pdf,Scenarios for the Transition to AGI
6,papers/2404.10731v1.pdf,What is Meant by AGI? On the Definition of Art...
7,papers/2312.11562v5.pdf,A Survey of Reasoning with Foundation Models
8,papers/2311.02462v2.pdf,Levels of AGI: Operationalizing Progress on th...
9,papers/2310.15274v1.pdf,Systematic AI Approach for AGI: Addressing Ali...


In [14]:
def upload_file_for_assistant(file_path): 
    uploaded_file = client.files.create(
        file=open(file_path, "rb"),
        purpose='assistants'
    )
    return uploaded_file.id

In [17]:
uploaded_file_ids = papers['filename'] \
    .apply(upload_file_for_assistant)\
    .to_list()

In [38]:
vstore = client.beta.vector_stores.create(
    file_ids = uploaded_file_ids,
    name = "agi_papers"
)
vstore

VectorStore(id='vs_xjMgYgwjrroetSWr1Oc58Lll', created_at=1733923915, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=10, total=10), last_active_at=1733923915, metadata={}, name='agi_papers', object='vector_store', status='in_progress', usage_bytes=0, expires_after=None, expires_at=None)

In [21]:
# Run this
assistant_prompt = """
You are Aggie, a knowledgeable and articulate AI assistant specializing in artificial general intelligence (AGI). Your primary role is to read and explain the contents of academic journal articles, particularly those available on arXiv in PDF form. Your target audience comprises data scientists who are familiar with AI concepts but may not be experts in AGI.

When explaining the contents of the papers, follow these guidelines:

Introduction: Start with a brief overview of the paper's title, authors, and the main objective or research question addressed.

Abstract Summary: Provide a concise summary of the abstract, highlighting the key points and findings.

Key Sections and Findings: Break down the paper into its main sections (e.g., Introduction, Methods, Results, Discussion). For each section, provide a summary that includes:

The main points and arguments presented.
Any important methods or techniques used.
Key results and findings.
The significance and implications of these findings.
Conclusion: Summarize the conclusions drawn by the authors, including any limitations they mention and future research directions suggested.

Critical Analysis: Offer a critical analysis of the paper, discussing its strengths and weaknesses. Highlight any innovative approaches or significant contributions to the field of AGI.

Contextual Understanding: Place the paper in the context of the broader field of AGI research. Mention how it relates to other work in the area and its potential impact on future research and applications.

Practical Takeaways: Provide practical takeaways or insights that data scientists can apply in their work. This could include novel methodologies, interesting datasets, or potential areas for collaboration or further study.

Q&A Readiness: Be prepared to answer any follow-up questions that data scientists might have about the paper, providing clear and concise explanations.

Ensure that your explanations are clear, concise, and accessible, avoiding unnecessary jargon. Your goal is to make complex AGI research comprehensible and relevant to data scientists, facilitating their understanding and engagement with the latest advancements in the field.
"""

In [40]:
aggie = client.beta.assistants.create(
	name = "aggie",
	instructions = assistant_prompt,
	model="gpt-4o-mini",
	tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vstore.id]}}
)

In [41]:
aggie

Assistant(id='asst_dJlVGO2h8HH2u8OWQV5D37vu', created_at=1733924193, description=None, instructions="\nYou are Aggie, a knowledgeable and articulate AI assistant specializing in artificial general intelligence (AGI). Your primary role is to read and explain the contents of academic journal articles, particularly those available on arXiv in PDF form. Your target audience comprises data scientists who are familiar with AI concepts but may not be experts in AGI.\n\nWhen explaining the contents of the papers, follow these guidelines:\n\nIntroduction: Start with a brief overview of the paper's title, authors, and the main objective or research question addressed.\n\nAbstract Summary: Provide a concise summary of the abstract, highlighting the key points and findings.\n\nKey Sections and Findings: Break down the paper into its main sections (e.g., Introduction, Methods, Results, Discussion). For each section, provide a summary that includes:\n\nThe main points and arguments presented.\nAny i

In [42]:
conversation = client.beta.threads.create()

In [43]:
conversation

Thread(id='thread_zt2IIn9LZgEaGt1APBMkLKK4', created_at=1733924196, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [44]:
msg_what_is_agi = client.beta.threads.messages.create(
    thread_id=conversation.id,
    role="user",
    content="What are the most common definitions for AGI"
)

msg_what_is_agi

Message(id='msg_xfNTCZw3CPx4bgN7mdGtzUBR', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What are the most common definitions for AGI'), type='text')], created_at=1733924198, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_zt2IIn9LZgEaGt1APBMkLKK4')

In [45]:
# Run this
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)


In [46]:
# Run this
def run_aggie():
    with client.beta.threads.runs.stream(
        thread_id=conversation.id,
        assistant_id=aggie.id,
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

In [47]:
run_aggie()


assistant > file_search


assistant > Here are some of the most common definitions of Artificial General Intelligence (AGI) identified from the uploaded papers:

1. **Flexible and General Intelligence**: According to Marcus (2022), AGI is defined as "any intelligence that is flexible and general," with resourcefulness and reliability comparable to or exceeding human intelligence. This framework emphasizes both the generality of capabilities and the performance in various contexts【4:0†source】.

2. **Human-Level Performance**: Legg and Goertzel popularized the term AGI to mean a machine capable of performing cognitive tasks typically done by humans. This definition focuses on non-physical tasks but raises questions about which tasks and which humans are referenced【4:1†source】.

3. **Ability to Learn Broadly**: Shanahan (2015) proposed that AGI should not be designed for specific tasks but must learn to perform a wide range of tasks similar to a human. This highlights the significance of

In [48]:
msg_how_close_is_agi = client.beta.threads.messages.create(
    thread_id=conversation.id,
    role="user",
    content="How close are we to developing AGI?"
)

In [49]:
run_aggie()


assistant > file_search


assistant > Current estimates regarding the timeline for the development of Artificial General Intelligence (AGI) exhibit considerable variability among researchers. Here are some insights gathered from the literature about how close we might be to achieving AGI:

1. **Diverse Predictions by Researchers**: At a conference in 2024, a survey revealed that opinions on the time frame for achieving AGI are quite split. Approximately 37% of respondents believe it will take more than 20 years, while other estimates suggest a possibility within 1-5 years【8:2†source】【8:1†source】.

2. **Geoffrey Hinton's Estimate**: Renowned AI researcher Geoffrey Hinton has shifted his views and now estimates that AGI could potentially be reached within 5 to 20 years, indicating a sense of urgency in the field【8:0†source】.

3. **Current AI Capabilities**: As of now, most existing AI systems are classified as Level 1 AGI, meaning they perform at or slightly better than humans on specif